# Intro to Data Wrangling and Tidy Data
*Author*: Zach del Rosario


### Learning outcomes
By working through this notebook, you will be able to:

- State the basic ideas of tidy data


In [ ]:
import numpy as np
import pandas as pd
import grama as gr

DF = gr.Intention()

# For downloading data
import os
import requests


The following code downloads the same data you extracted in the previous day's Tabula exercise.


In [ ]:
# Filename for local data
filename_data = "./data/tabula-weibull.csv"

# The following code downloads the data, or (after downloaded)
# loads the data from a cached CSV on your machine
if not os.path.exists(filename_data):
    # Make request for data
    url_data = "https://raw.githubusercontent.com/zdelrosario/mi101/main/mi101/data/tabula-weibull1939-table4.csv"
    r = requests.get(url_data, allow_redirects=True)
    open(filename_data, 'wb').write(r.content)
    print("   Tabula-extracted data downloaded from public Google sheet")
else:
    # Note data already exists
    print("   Tabula-extracted data loaded locally")
    
# Read the data into memory
df_tabula = pd.read_csv(filename_data)


These are data on the tensile strength of specimens of stearic acid and plaster-of-paris.


In [ ]:
df_tabula


Note that Pandas renamed some columns to avoid giving us duplicate column names. The names are useful for holding metadata, but shorter column names are far easier to work with in a computational environment.


### __Q1__: Complete the code below to rename the columns

*Hint*: You can click-and-drag on the DataFrame printout above for a less error-prone way of giving the original column names.


In [ ]:
###
# TASK: Copy the original column names into the double-quote below
#       to complete the code and rename the columns with shorter
#       names.
###

df_q1 = (
    df_tabula
    >> gr.tf_rename(
## TASK: Uncomment these lines and fill-in the new (short) column names
#         obs_1="",   # Observation Number, Block 1
#         area_1="",  # Specimen area, Block 1
#         sigma_1="", # Stress, Block 1
#         obs_2="",   # Observation Number, Block 2
#         area_2="",  # Specimen area, Block 2
#         sigma_2="", # Stress, Block 2

    )
)

## NOTE: No need to edit, this will show your renamed data
df_q1

Use the following to check your work:


In [ ]:
## NO NEED TO EDIT; use this to check your work
assert(set(df_q1.columns) == {"obs_1", "area_1", "sigma_1", "obs_2", "area_2", "sigma_2"})
print("Success!")


Now the column names are much shorter, but we've


### __Q2__: Complete the *data dictionary* below to document the units associated with the short column names.

*Note*: Weibull in his (1939) paper reports these stress values in units `kg / mm^2`. For the data to be sensible, his `kg` must refer to a kilogram-force, sometimes denoted `kgf`. One `kgf` is the force exerted by a kilogram in standard gravity (`g = 9.8 m/s^2`). We'll convert to less strange units later!


In [ ]:
df_tabula.head()



| Column | Units |
|--------|-------|
| `obs_1`   | (Unitless) |
| `area_1`, | ??? |
| `sigma_1` | ??? |
| `obs_2`   | ??? |
| `area_2`, | ??? |
| `sigma_2` | ??? |



## Pivoting

(TODO Describe the "blocks")

In [ ]:
df_q1.head()


Imagine we wanted to compute some simple statistics on these data, say the mean of the stress values. Since the data come in two blocks, we have to access them separately:

In [ ]:
sigma_mean_1 = df_q1.sigma_1.mean()
sigma_mean_2 = df_q1.sigma_2.mean()

print("Mean 1: {0:4.3f}".format(sigma_mean_1))
print("Mean 2: {0:4.3f}".format(sigma_mean_2))

We could do something hacky to combine the two:


In [ ]:
sigma_mean_both = (12 * sigma_mean_1 + 11 * sigma_mean_2) / (12 + 11)
print("Mean both: {0:4.3f}".format(sigma_mean_both))

But it would be far easier if we could just *combine* all the relevant columns so they're not in "blocks." This is what **pivoting** a dataset allows us to do:


In [ ]:
## NOTE: NO need to edit; you'll learn how to do this in the evening's notebook
df_long = (
    df_q1
    >> gr.tf_pivot_longer(
        columns=["obs_1", "area_1", "sigma_1", "obs_2", "area_2", "sigma_2"],
        names_to=[".value", "block"],
        names_sep="_",
    )
    >> gr.tf_arrange(DF.obs)
)
df_long

## Tidy Data


(TODO: Write up the principles of tidy data.)


## Data Wrangling

(Part of data wrangling is screaming. Here's an example trying to make sense of the plaster data.)


Weibull reports the $\sigma_d$ values in `kg / mm^2`; if we interpret `kg` as a kilogram (mass) then these can't be stress values! However, suppose for a moment that he were using `kg` to denote a kilogram-force, where $1 \text{kgf} = 1 \text{kg} \times 9.8 m/s^2$.

Weibull gives a summary value for the same stress in the more interpretable units `g / (cm s^2)`. Let's check this hypothesis by comparing the proposed unit converstion with our data:


In [ ]:
# (540 x 10^5 g/(cm s^2)) / (980 cm/s^2) * (kg / 1000 g) * (cm^2 / 100 mm^2)
540e5 / 980 / 1000 / 100


This is very near the `sigma` values we have in our dataset, which lends a great deal of credibility to our interpretation of `kg` as `kgf`. With this, we can make a unit conversation to more standard units.

$$\text{kgf} / \text{mm}^2 = 9.8 \text{MPa}$$


### __Q3__: Convert the units to MPa.


In [ ]:
###
# TASK: Replace the 1.0 factor with the correct conversion factor
###


df_q3 = (
    df_long
    >> gr.tf_mutate(sigma_MPa=DF.sigma * 1.0) # Replace 1.0 with correct factor

)

df_q3.head()